<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/MaximizarInvestimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maximizar Investimentos**
---
$$ maximize \sum\limits_{i=1}^m l_i.x_i$$ 


$$s.\ a.\; \sum\limits_{i=1}^m p_i.x_i\leq C $$

$$ x_j \in \{0, 1\},\; j = 1, ... m $$

$ Onde:\\
c - Custo\ de\ cada\ item;\\
C - Custo\ máxima\ do\ investidor\\
l - Retorno\ de\ cada\ item;\\
x - Variácel\ binária ;\\
i - itens.\\
m - total\ de\ itens\\
$

In [ ]:
pip install pyomo -q

In [ ]:
!apt-get install -y -qq glpk-utils -q
!apt-get install -y -qq coinor-cbc -q

In [3]:
#Instalar bibliotes necessárias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pyomo.environ as pyo

In [22]:
#Ler dados~
link = '/content/dados.xlsx'
df = pd.read_excel(link)
print("Número de colunas", df.shape[1])
print("Dimensão do problema", df.shape[0])
df

Número de colunas 3
Dimensão do problema 25


,Variável,Investimento,Retorno
0,P1,2583.177243,287.089823
1,P2,9625.520779,3110.447724
2,P3,1917.758549,206.947104
3,P4,5610.365463,2426.473442
4,P5,2132.106199,2078.517159
5,P6,9348.902294,5645.969243
6,P7,5761.805555,2865.621342
7,P8,740.482288,152.000865
8,P9,3328.875892,2284.589824
9,P10,103.955427,33.050690


In [25]:
print("Preço total: R$", np.round(df.Investimento.sum(),2))

Preço total: R$ 113564.79


In [26]:
#Informação sobre os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Variável      25 non-null     object 
 1   Investimento  25 non-null     float64
 2   Retorno       25 non-null     float64
dtypes: float64(2), object(1)
memory usage: 728.0+ bytes


In [27]:
#Sumarização estatistica dos dados
df.describe()

,Investimento,Retorno
count,25.000000,25.000000
mean,4542.591726,2524.374097
std,2823.441949,2183.726902
min,103.955427,33.050690
25%,2132.106199,714.608805
50%,4609.794472,2426.473442
75%,6315.193176,3198.078147
max,9625.520779,7882.748627


In [60]:
#Variáveis de Decisão
C = 2000
m = df.shape[0]  #Dimensão do Problema
custo = list(df.Investimento) #Transformar valores em lista
lucro = list(df.Retorno)   #Transformar pesos em lista
variaveis = list(df.Variável)

In [61]:
# 1 Declaração do modelo
model = pyo.ConcreteModel()

# 2 Declarar as variáveis de decisão e parâmetros
model.I = pyo.RangeSet(m)
model.x = pyo.Var(model.I, within = pyo.Binary)
model.c = pyo.Param(model.I, initialize = lambda model, i: custo[i -1])
model.l = pyo.Param(model.I, initialize = lambda model, i: lucro[i -1])

# 3 Declarar a função objetivo
model.obj = pyo.Objective(expr = sum(model.l[i]*model.x[i] for i in model.I), sense = pyo.maximize)

# 4 Declarar as restrições
model.c1 = pyo.Constraint(expr = sum(model.c[i]*model.x[i] for i in model.I) <= C)

In [62]:
print(model.pprint())

1 RangeSet Declarations
    I : Dimen=1, Size=25, Bounds=(1, 25)
        Key  : Finite : Members
        None :   True :  [1:25]

2 Param Declarations
    c : Size=25, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  2583.177243313424
          2 :  9625.520778540857
          3 : 1917.7585485362235
          4 :  5610.365462936991
          5 :  2132.106199109146
          6 :  9348.902294488375
          7 :  5761.805554703392
          8 :  740.4822884176132
          9 :  3328.875892428994
         10 : 103.95542709151329
         11 : 1832.8458199417385
         12 : 3495.9085931414156
         13 :  3109.145469042762
         14 :  8373.209993257988
         15 : 1344.3008041058447
         16 :  977.3476057301789
         17 :  2937.043001938584
         18 :  5260.537226092384
         19 :  6683.871587878048
         20 :  8319.534869881878
         21 :  5170.345490443773
         22 :   6315.19317646058
         23 :  7992.898812025239
    

In [63]:
#Declarar o Solver
solver_glpk = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')

In [64]:
#Resolver o problema com glpk
result = solver_glpk.solve(model)

In [65]:
model.pprint()

1 RangeSet Declarations
    I : Dimen=1, Size=25, Bounds=(1, 25)
        Key  : Finite : Members
        None :   True :  [1:25]

2 Param Declarations
    c : Size=25, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  2583.177243313424
          2 :  9625.520778540857
          3 : 1917.7585485362235
          4 :  5610.365462936991
          5 :  2132.106199109146
          6 :  9348.902294488375
          7 :  5761.805554703392
          8 :  740.4822884176132
          9 :  3328.875892428994
         10 : 103.95542709151329
         11 : 1832.8458199417385
         12 : 3495.9085931414156
         13 :  3109.145469042762
         14 :  8373.209993257988
         15 : 1344.3008041058447
         16 :  977.3476057301789
         17 :  2937.043001938584
         18 :  5260.537226092384
         19 :  6683.871587878048
         20 :  8319.534869881878
         21 :  5170.345490443773
         22 :   6315.19317646058
         23 :  7992.898812025239
    

In [66]:
# Resultado do modelo
print(result)
print(model.obj())


Problem: 
- Name: unknown
  Lower bound: 750.183316509329
  Upper bound: 750.183316509329
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 26
  Number of nonzeros: 26
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.02769184112548828
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

750.1833165093292


In [74]:
for idx, value in enumerate(variaveis):
    print(value, "->", model.x[idx+1](), '->', model.c[idx+1])

P1 -> 0.0 -> 2583.177243313424
P2 -> 0.0 -> 9625.520778540857
P3 -> 0.0 -> 1917.7585485362235
P4 -> 0.0 -> 5610.365462936991
P5 -> 0.0 -> 2132.106199109146
P6 -> 0.0 -> 9348.902294488375
P7 -> 0.0 -> 5761.805554703392
P8 -> 0.0 -> 740.4822884176132
P9 -> 0.0 -> 3328.875892428994
P10 -> 1.0 -> 103.95542709151329
P11 -> 0.0 -> 1832.8458199417385
P12 -> 0.0 -> 3495.9085931414156
P13 -> 0.0 -> 3109.145469042762
P14 -> 0.0 -> 8373.209993257988
P15 -> 1.0 -> 1344.3008041058447
P16 -> 0.0 -> 977.3476057301789
P17 -> 0.0 -> 2937.043001938584
P18 -> 0.0 -> 5260.537226092384
P19 -> 0.0 -> 6683.871587878048
P20 -> 0.0 -> 8319.534869881878
P21 -> 0.0 -> 5170.345490443773
P22 -> 0.0 -> 6315.19317646058
P23 -> 0.0 -> 7992.898812025239
P24 -> 0.0 -> 5989.866530273771
P25 -> 0.0 -> 4609.794471946614
